In [ ]:
"""ESTABLISH A CONNECTION WITH THE DEVICES"""
import vna
IP = "192.168.3.51"
timeout = 25e3
max_npoints = 1e3
instr_vna = vna.ManageVNA(IP,timeout,max_npoints)
import sma
resource = 'TCPIP0::192.168.3.2::INSTR'
instr_sma = sma.ManageSMA(resource)
import sim
resource = 'ASRL34::INSTR'
instr_sim = sim.ManageSIM(resource)

In [ ]:
import time
import numpy as np
from many_instruments import *
import h5py

In [ ]:
"""If the Bias stops responding, you should close the connection and reopen it, it seems to work"""
def reopen_sim(instr_sim):
    instr_sim.close()
    time.sleep(4)
    resource = 'ASRL34::INSTR'
    new_instr = sim.ManageSIM(resource)
    time.sleep(4)
    return new_instr

In [ ]:
"""Prepare VNA"""
port = "S21"
IFBW = 1e2 #[Hz]
powerdBm = -15 #[dBm]

instr_vna.reset()
instr_vna.set_mode("NA")
time.sleep(1)
instr_vna.set_port(port)
time.sleep(1)
instr_vna.set_IFBW(IFBW)

In [ ]:
"""SCAN (VNA) parameters """
# The scan params are specified in each dataset name

fmin = 1e9
fmax = 14e9
npoints = 401
navgs = 1

In [ ]:
"""Choose where SAVE results"""
filename = "10_03_DOPO_RISC.h5"

In [ ]:
"""Simulation parameters"""
# Pump
freq_start = 15.5e9
freq_stop = 17e9

ampl_min = -20
ampl_max = -20
# Bias
R = 434 #[ +-1 Ohm ] #Calculated with multimeter and verifiyed measuring current with amperometer
i_min = 0.76e-3
i_max = 0.76e-3
volt_min = R*i_min
volt_max = R*i_max

# Points densities
n_freq = 20
n_ampl = 1
n_volt = 1

freqs = np.linspace(freq_start, freq_stop, n_freq) 
ampls = np.linspace(ampl_min, ampl_max, n_ampl)
volts = np.linspace(volt_min, volt_max, n_volt)


#volts = [0.227, 0.277, 0.327, 0.377, 0.427]
#n_volt = len(volts)
#instr_vna.set_range(10000000000,14000000000)

In [ ]:
"""Save simulation parameters in a dataset 'parameters' """
name_array = ["powerdBm","freq_start","freq_stop","ampl_min","ampl_max","volt_min","volt_max","n_freq","n_ampl","n_volt","navgs","R"]
value_array = [str(powerdBm),str(freq_start),str(freq_stop),str(ampl_min),str(ampl_max),str(volt_min),str(volt_max),str(n_freq),str(n_ampl),str(n_volt),str(navgs),str(R)]
hf = h5py.File(filename, 'a')
mat = [name_array, value_array]
hf.create_dataset("parameters", data = mat)
hf.close()

In [ ]:
"""Save a measure with all the devices OFF"""
### Save the 'Off' measure
# Turn Off Pump and Bias
instr_sma.set_output(0)
try:
    out = instr_sim.set_output(0)
except:
    out = instr_sim.set_output(0)
    instr_sim = reopen_sim(instr_sim)
if out != "0":
    print("Bias not responding!")
    instr_sim = reopen_sim(instr_sim)
    instr_sim.set_output(0)

acquire_S21(instr_vna,fmin,fmax,powerdBm,npoints,navgs,filename,"Off")

### Turn on the SMA (Pump)
# Before we set 'safe' values
instr_sma.set_freq(16e9)
instr_sma.set_amplitude(-30)
instr_sma.set_output(1)

## Turn on the SIM (Bias)

try:
    out = instr_sim.set_output(1)
except:
    instr_sim = reopen_sim(instr_sim)
    out = instr_sim.set_output(1)
if out != "1":
    print("Bias not responding!")
    instr_sim = reopen_sim(instr_sim)
    instr_sim.set_output(1)

In [ ]:
"""SIMULATION"""
wait_meter = 0.1    # Time (in seconds) the code waits between changing a parameter and making the successive measure
sim_out_volt = ""
sim_out_output = ""
for k in np.arange(n_volt) :
    time.sleep(wait_meter)
    try:
        instr_sim.set_voltage(round(volts[k],3))

        # Ask actual voltage
        if instr_sim.get_voltage() != str(round(volts[k],3)):
            print("Cannot change Bias voltage")
            instr_sim = reopen_sim(instr_sim)
            # Ask actual voltage
            instr_sim.set_voltage(round(volts[k],3))
    except:
        instr_sim = reopen_sim(instr_sim)
        instr_sim.set_voltage(round(volts[k],3))
        
        try:
            # Ask actual voltage
            if instr_sim.get_voltage() != str(round(volts[k],3)):
                print("Cannot change Bias voltage")
                instr_sim = reopen_sim(instr_sim)
                # Ask actual voltage
                instr_sim.set_voltage(round(volts[k],3))
        except:
            instr_sim = reopen_sim(instr_sim)
            if instr_sim.get_voltage() != str(round(volts[k],3)):
                print("Cannot change Bias voltage")
                instr_sim = reopen_sim(instr_sim)
                # Ask actual voltage
                instr_sim.set_voltage(round(volts[k],3))
        
    for j in np.arange(n_ampl) :
        time.sleep(wait_meter)
        instr_sma.set_amplitude(ampls[j])
        for i in np.arange(n_freq) :
            time.sleep(wait_meter)
            instr_sma.set_freq(freqs[i])
            group = "dc_" + str(k) + "/pumpfreq_" + str(i) + "/pumppow_" + str(j)
            #freq, I, Q = instr_vna.single_scan(fmin,fmax,powerdBm,npoints,navgs)
            acquire_S21(instr_vna,fmin,fmax,powerdBm,npoints,navgs,filename,group)
            

            # #Write
            # Mat = [freq,I,Q]
            # dataset_name = "dataset_" + str(index)
            # hf.create_dataset(dataset_name, data=Mat)
            # index = index + 1

"""Turn OFF Pump and Bias"""
instr_sma.set_output(0) # Pump Off


try:
    out = instr_sim.set_output(0)
except:
    instr_sim = reopen_sim(instr_sim)
    out = instr_sim.set_output(0)
if out != "0":
    print("Bias not responding!")
    instr_sim = reopen_sim(instr_sim)
    instr_sim.set_output(0)

In [ ]:
"""Save a measure with all the devices OFF"""
### Save the 'Off' measure 
acquire_S21(instr_vna,fmin,fmax,powerdBm,npoints,navgs,filename,"Off_end")

In [ ]:
instr_sim.close()
instr_sma.close()
instr_vna.close()